In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_mch1
(
  mch1_id STRING,
  mch1_name STRING,
  mch1_desc STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
 
""")

In [0]:

spark.sql(f"""CREATE OR REPLACE TEMP VIEW temp_d_mch1 AS
SELECT
a._bic_zc_mchlv1 mch1_id,  
a.txtsh mch1_name,
a.txtmd mch1_desc
FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY _bic_zc_mchlv1 ORDER BY proc_date DESC) rn 
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_zc_mchlv1
        WHERE end_date = '2400-01-01'
    ) AS a WHERE rn = 1 
;""")

In [0]:
spark.sql(
    f"""MERGE INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_mch1 AS tgt
USING temp_d_mch1 AS src
ON tgt.mch1_id = src.mch1_id

WHEN MATCHED THEN 
    UPDATE SET 
        tgt.mch1_name = src.mch1_name,
        tgt.mch1_desc = src.mch1_desc

WHEN NOT MATCHED THEN 
    INSERT (mch1_id, mch1_name, mch1_desc)
    VALUES (src.mch1_id, src.mch1_name, src.mch1_desc);
""")

In [0]:
spark.sql(f"DROP VIEW IF EXISTS temp_d_mch1")